In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, when, udf, to_date
from pyspark.sql.types import IntegerType

spark = SparkSession.builder \
    .appName("Music Data Cleaning") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

# Carregue seu arquivo parquet
df = spark.read.parquet("/spark-data/meuarquivo.parquet")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/08 05:46:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
                                                                                

In [2]:
from pyspark.sql.functions import col, lower, split, substring

# Vamos imprimir a contagem inicial de linhas para comparação
initial_count = df.count()
print(f"Número inicial de linhas: {initial_count}")

# --- Passo 1: Limpar dados duplicados ---
# Remove linhas onde a combinação de artista, álbum e música é idêntica.
df_no_duplicates = df.dropDuplicates(['Artist(s)', 'Album', 'song'])
print(f"Linhas após remover duplicatas: {df_no_duplicates.count()}")

Número inicial de linhas: 551443


[Stage 6:=============================>                             (1 + 1) / 2]

Linhas após remover duplicatas: 498052


In [3]:
# --- Passo 2: Tratar dados nulos ---
# Remove linhas onde colunas essenciais como 'song', 'Album' ou 'Time signature' são nulas.
df_no_nulls = df_no_duplicates.na.drop(subset=['song', 'Album', 'Time signature'])
print(f"Linhas após remover nulos: {df_no_nulls.count()}")

[Stage 12:=============================>                            (1 + 1) / 2]

Linhas após remover nulos: 498021


In [4]:
# --- Passo 3: Limpar e transformar colunas ---
df_transformed = df_no_nulls.withColumn(
    # Deixar 'Release Date' apenas com o ano
    "release_year",
    substring(col("Release Date"), -4, 4).cast("int")
).withColumn(
    # Deixar 'Genre' apenas com o primeiro valor
    "main_genre",
    split(col("Genre"), ",")[0]
).withColumn(
    # Descaptalizar os valores da coluna 'emotion'
    "emotion_lower",
    lower(col("emotion"))
)

In [5]:
# --- Passo 4: Apagar emoções com poucos valores ---
# Lista de emoções a serem removidas (já em minúsculas)
emotions_to_remove = ['true', 'pink', 'thirst', 'angry', 'confusion', 'interest']

df_filtered = df_transformed.filter(
    ~col("emotion_lower").isin(emotions_to_remove)
)
print(f"Linhas após filtrar emoções: {df_filtered.count()}")

[Stage 18:=============================>                            (1 + 1) / 2]

Linhas após filtrar emoções: 497998


In [6]:
# --- Passo 5: Apagar colunas que não vamos usar ---
columns_to_drop = [
    # Colunas originais que foram transformadas
    'Release Date', 'Genre', 'emotion',
    # Colunas que o usuário pediu para apagar
    'song',
    'Similar Artist 1', 'Similar Song 1', 'Similarity Score 1',
    'Similar Artist 2', 'Similar Song 2', 'Similarity Score 2',
    'Similar Artist 3', 'Similar Song 3', 'Similarity Score 3'
]

df_final = df_filtered.drop(*columns_to_drop)

In [7]:
# --- Passo 5: Apagar colunas que não vamos usar ---
columns_to_drop = [
    # Colunas originais que foram transformadas
    'Release Date', 'Genre', 'emotion',
    # Colunas que o usuário pediu para apagar
    'song',
    'Similar Artist 1', 'Similar Song 1', 'Similarity Score 1',
    'Similar Artist 2', 'Similar Song 2', 'Similarity Score 2',
    'Similar Artist 3', 'Similar Song 3', 'Similarity Score 3'
]

df_final = df_filtered.drop(*columns_to_drop)


# --- Resultado Final ---
print("\nEsquema do DataFrame final:")
df_final.printSchema()

print("\nAmostra dos dados limpos e pré-processados:")
df_final.show(10)


Esquema do DataFrame final:
root
 |-- Artist(s): string (nullable = true)
 |-- text: string (nullable = true)
 |-- Length: string (nullable = true)
 |-- Album: string (nullable = true)
 |-- Key: string (nullable = true)
 |-- Tempo: string (nullable = true)
 |-- Loudness (db): string (nullable = true)
 |-- Time signature: string (nullable = true)
 |-- Explicit: string (nullable = true)
 |-- Popularity: string (nullable = true)
 |-- Energy: string (nullable = true)
 |-- Danceability: string (nullable = true)
 |-- Positiveness: string (nullable = true)
 |-- Speechiness: string (nullable = true)
 |-- Liveness: string (nullable = true)
 |-- Acousticness: string (nullable = true)
 |-- Instrumentalness: string (nullable = true)
 |-- Good for Party: string (nullable = true)
 |-- Good for Work/Study: string (nullable = true)
 |-- Good for Relaxation/Meditation: string (nullable = true)
 |-- Good for Exercise: string (nullable = true)
 |-- Good for Running: string (nullable = true)
 |-- Good fo

25/07/08 05:46:44 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

+--------------------+--------------------+------+--------------------+------+-----+-------------+--------------+--------+----------+------+------------+------------+-----------+--------+------------+----------------+--------------+-------------------+------------------------------+-----------------+----------------+------------------------+----------------+--------------------------+------------------------+------------+----------+-------------+
|           Artist(s)|                text|Length|               Album|   Key|Tempo|Loudness (db)|Time signature|Explicit|Popularity|Energy|Danceability|Positiveness|Speechiness|Liveness|Acousticness|Instrumentalness|Good for Party|Good for Work/Study|Good for Relaxation/Meditation|Good for Exercise|Good for Running|Good for Yoga/Stretching|Good for Driving|Good for Social Gatherings|Good for Morning Routine|release_year|main_genre|emotion_lower|
+--------------------+--------------------+------+--------------------+------+-----+-------------+

In [8]:
# Supondo que seu DataFrame final e limpo se chama 'df_final'

# 1. Defina o caminho de saída para o novo dataset
caminho_saida = "/spark-data/musicas_limpas_cluste.parquet"

# 2. Salve o DataFrame no formato Parquet
df_final.write.mode("overwrite").parquet(caminho_saida)

print(f"DataFrame limpo foi salvo com sucesso em: {caminho_saida}")

25/07/08 05:47:40 WARN TaskSetManager: Lost task 0.0 in stage 27.0 (TID 36) (10.0.1.6 executor 0): ExitCodeException exitCode=1: chmod: changing permissions of '/spark-data/musicas_limpas_cluste.parquet/_temporary/0/_temporary': Operation not permitted

	at org.apache.hadoop.util.Shell.runCommand(Shell.java:1007)
	at org.apache.hadoop.util.Shell.run(Shell.java:900)
	at org.apache.hadoop.util.Shell$ShellCommandExecutor.execute(Shell.java:1212)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:1306)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:1288)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermissi

Py4JJavaError: An error occurred while calling o69.parquet.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 27.0 failed 4 times, most recent failure: Lost task 0.3 in stage 27.0 (TID 42) (10.0.1.6 executor 0): ExitCodeException exitCode=1: chmod: changing permissions of '/spark-data/musicas_limpas_cluste.parquet/_temporary/0/_temporary/attempt_202507080547398618919613725951368_0027_m_000000_42': Operation not permitted

	at org.apache.hadoop.util.Shell.runCommand(Shell.java:1007)
	at org.apache.hadoop.util.Shell.run(Shell.java:900)
	at org.apache.hadoop.util.Shell$ShellCommandExecutor.execute(Shell.java:1212)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:1306)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:1288)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.m